In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from package.utils import DotDict, timer, get_config, load_data, merge_data

In [5]:
%%time
bucket_path = 's3://tli-crm-segmentation/data-zones/neutral/raw'
dtypes = {'policyno':str, 'party_id_name_bd':str, 'party_id_govt':str, 'policy_govt':str}

path = f'{bucket_path}/stg_tb_mstperson_beneficiary_party_id_name_bd_party_id_govt_202505251050.csv'
beneid_df = pd.read_csv(path, dtype=dtypes)

# path = f'{bucket_path}/stg_tb_policy_policyparticipant_202505220106.csv'
# policybene_df = pd.read_csv(path, dtype=dtypes)

CPU times: user 15.5 s, sys: 3.47 s, total: 18.9 s
Wall time: 35.9 s


In [7]:
beneid_df.policytype.unique()

array(['O', 'U', 'W', 'I', 'L'], dtype=object)

In [18]:
beneid_df.isna().sum()

policytype                 0
policyno                   0
relationshipcode           0
party_id_name_bd    18989768
party_id_govt       14758009
dtype: int64

In [11]:
path = 's3://dev-tli-eap-inbound-bucket/test2_202505221645.csv'
dtypes = {'policyno':str, 'party_id_name_bd':str, 'party_id_govt':str, 'policy_govt':str}
policybene_df = pd.read_csv(path, dtype=dtypes)

mask = policybene_df['party_id_govt'].isna()
mask &= ~policybene_df['dup_party_id_govt'].isna()
policybene_df[mask]['dup_party_id_govt'].nunique()

In [13]:
policybene_df.shape

(17786852, 7)

4

In [28]:
policybene_df.drop_duplicates(subset='party_id_govt')['policyno'].nunique()

1399884

In [29]:
policybene_df['policyno'].nunique()

12450848

In [23]:
policybene_df[mask]['dup_party_id_govt'].value_counts()

365443.0    12399450
3.0           601458
9.0               22
6.0                4
Name: dup_party_id_govt, dtype: int64

In [1]:
# path = f'{bucket_path}/stg_common_relationship_202501220951.csv'
# benedict_df = pd.read_csv(path)

ทำ partyid, policyno เป็น str ด้วย

In [27]:
bene_list = ['บิดา', 'บิดาบุญธรรม', 'มารดา', 'มารดาบุญธรรม', 'บุตร', 'บุตรบุญธรรม', 'สามี', 'ภรรยา']
mask = benedict_df['relationship_desc_th'].isin(bene_list)
benecode_list = set(benedict_df.loc[mask, 'relationship_cd'])

In [17]:
benecode_list

{1, 2, 3, 4, 5, 22, 43, 44}

In [31]:
beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_govt'] = beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_name_bd']
mask = ~beneid_df['party_id_govt'].isna()
mask &= beneid_df['relationshipcode'].isin([1, 2, 3, 4, 5, 22, 43, 44])
beneid_df = beneid_df.loc[mask, :]
beneid_df.head()

,policytype,policyno,relationshipcode,party_id_name_bd,party_id_govt
850874,O,39925822,3,NaN,10823783
850875,O,39926340,1,NaN,10823783
850876,O,39925822,1,NaN,10823783
850878,O,86716784,4,NaN,10823714
850879,O,87036551,1,NaN,10823714


In [36]:
beneid_df.drop_duplicates(subset='policyno')

,policytype,policyno,relationshipcode,party_id_name_bd,party_id_govt
850874,O,39925822,3,NaN,10823783
850875,O,39926340,1,NaN,10823783
850878,O,86716784,4,NaN,10823714
850879,O,87036551,1,NaN,10823714
850880,O,86833981,4,NaN,10823714
...,...,...,...,...,...
18989748,O,37410894,3,NaN,10505282
18989749,O,23109183,4,NaN,11221824
18989754,O,80227109,5,NaN,10505306
18989756,O,80232119,1,NaN,10505308


In [4]:
policybene_df.head()

,party_id,policy_govt
0,4596828,20604519
1,2060465,20604684
2,2468579,20605089
3,3455457,20605591
4,3328568,20607297


In [38]:
policybene_df.shape

(17963767, 2)

In [ ]:
eneid_df.drop_duplicates(subset='policyno')

In [ ]:
policyactive_list = set(policy_df['policyno'])
policybene_df = 

In [ ]:
import pandas as pd
from package.utils import timer, load_data

@timer
def createfamilyflag_pipeline(policy_df, policybene_path, beneid_path):
    dtypes = {'policyno':str, 'party_id_name_bd':str, 'party_id_govt':str, 'policy_govt':str, 'party_id':str}
    policybene_df = load_data(policybene_path, dtype=dtypes)
    beneid_df = load_data(beneid_path, dtype=dtypes)
    
    # get active policyno
    policyactive_list = set(policy_df['policy_no'])
    
    # get active beneficiary partyid
    mask = policybene_df['policy_govt'].isin(policyactive_list)
    activebeneid_list = set(policybene_df.loc[mask, 'party_id'])
    
    # get policy that having active beneficiary
    beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_govt'] = beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_name_bd']
    mask = ~beneid_df['party_id_govt'].isna()
    mask &= beneid_df['relationshipcode'].isin([1, 2, 3, 4, 5, 22, 43, 44])
    mask &= beneid_df['policyno'].isin(policyactive_list)
    mask &= beneid_df['party_id_govt'].isin(activebeneid_list)
    familypolicyno_list = set(beneid_df.loc[mask, 'policyno'])
    
    # get df having party_rk and family flag
    proxy = policy_df[['party_rk', 'policy_no']].copy()
    proxy['is_family'] = 0
    proxy.loc[proxy['policy_no'].isin(familypolicyno_list), 'is_family'] = 1
    proxy = proxy[['party_rk', 'is_family']].sort_values(by='is_family', ascending=False)
    proxy = proxy.drop_duplicates(subset='party_rk', keep='first')
    print(proxy)
    return proxy

createfamilyflag_pipeline(policy_df, policybene_path, beneid_path)

In [31]:
import pandas as pd
from package.utils import timer, load_data

@timer
def createfamilyflag_pipeline(policy_df, policybene_df, beneid_df):
    # dtypes = {'policyno':str, 'party_id_name_bd':str, 'party_id_govt':str, 'policy_govt':str, 'dup_party_id_govt':str}
    # policybene_df = load_data(policybene_path, dtype=dtypes)
    # beneid_df = load_data(beneid_path, dtype=dtypes)
    
    # get active policyno
    policyactive_list = set(policy_df['policy_no'])
    print(policy_df['policy_no'])
    
    # get active beneficiary partyid
    mask = policybene_df['policy_govt'].isin(policyactive_list)
    activebeneid_list = set(policybene_df.loc[mask, 'party_id'])
    print(policybene_df.loc[mask, 'party_id'])
    
    # get policy that having active beneficiary
    beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_govt'] = beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_name_bd']
    mask = ~beneid_df['party_id_govt'].isna()
    mask &= beneid_df['relationshipcode'].isin([1, 2, 3, 4, 5, 22, 43, 44])
    mask &= beneid_df['policyno'].isin(policyactive_list)
    mask &= beneid_df['party_id_govt'].isin(activebeneid_list)
    beneid_df = beneid_df.loc[mask, :]
    print(beneid_df)
    # beneid_df = beneid_df.drop_duplicates(subset='party_id_govt')
    familypolicyno_list = set(beneid_df['policyno'])
    
    # get df having party_rk and family flag
    proxy = policy_df[['party_rk', 'policy_no']].copy()
    proxy['is_family'] = 0
    proxy.loc[proxy['policy_no'].isin(familypolicyno_list), 'is_family'] = 1
    proxy = proxy[['party_rk', 'is_family']].sort_values(by='is_family', ascending=False)
    proxy = proxy.drop_duplicates(subset='party_rk', keep='first')
    print(proxy)
    return proxy

In [3]:
bucket_path = 's3://tli-crm-segmentation/data-test/family'
policybene_path = f'{bucket_path}/stg_tb_policy_policyparticipant_202505220106.csv'
# beneid_path = f'{bucket_path}/stg_tb_mstperson_beneficiary_party_id_name_bd_party_id_govt_202505220042.csv'
beneid_path = 's3://dev-tli-eap-inbound-bucket/test2_202505221645.csv'

dtypes = {'policy_no':str, 'party_id_name_bd':str, 'party_id_govt':str, 'policy_govt':str, 'dup_party_id_govt':str, 'party_rk':str, 'party_id':str}
policy_df = pd.read_csv('active_policy.csv', dtype=dtypes)
policybene_df = load_data(policybene_path, dtype=dtypes)
beneid_df = load_data(beneid_path, dtype=dtypes)

function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-test/family/stg_tb_policy_policyparticipant_202505220106.csv


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:286: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Succesfully loaded data from: s3://tli-crm-segmentation/data-test/family/stg_tb_policy_policyparticipant_202505220106.csv
function: load_data successfully executed at 21.34773325920105s
function: load_data is starting...
loading data from: s3://dev-tli-eap-inbound-bucket/test2_202505221645.csv


FileNotFoundError: dev-tli-eap-inbound-bucket/test2_202505221645.csv

In [92]:
bucket_path = 's3://tli-crm-segmentation/data-test/family'
policybene_path = f'{bucket_path}/stg_tb_policy_policyparticipant_202505220106.csv'
beneid_path = f'{bucket_path}/stg_tb_mstperson_beneficiary_party_id_name_bd_party_id_govt_202505220042.csv'
# beneid_path = 's3://dev-tli-eap-inbound-bucket/test2_202505221645.csv'

dtypes = {'policy_no':str, 'party_id_name_bd':str, 'party_id_govt':str, 'policy_govt':str, 'dup_party_id_govt':str, 'party_rk':str, 'party_id':str}
policy_df = pd.read_csv('active_policy.csv', dtype=dtypes)
policybene_df = load_data(policybene_path, dtype=dtypes)
beneid_df = load_data(beneid_path, dtype=dtypes)

policyactive_list = set(policy_df['policy_no'])
print(policy_df['policy_no'])

mask = policybene_df['policy_govt'].isin(policyactive_list)
activebeneid_list = set(policybene_df.loc[mask, 'party_id'])
print(policybene_df.loc[mask, 'party_id'])

beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_govt'] = beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_name_bd']
mask = ~beneid_df['party_id_govt'].isna()
mask &= beneid_df['relationshipcode'].isin([1, 2, 3, 4, 5, 22, 43, 44])
mask &= beneid_df['policyno'].isin(policyactive_list)
mask &= beneid_df['party_id_govt'].isin(activebeneid_list)
beneidactive_df = beneid_df.loc[mask, :].head()
print(beneidactive_df.shape)
familypolicyno_list = set(beneidactive_df['policyno'])

# get df having party_rk and family flag
proxy = policy_df[['party_rk', 'policy_no']].copy()
proxy['is_family'] = 0
proxy.loc[proxy['policy_no'].isin(familypolicyno_list), 'is_family'] = 1
proxy = proxy[['party_rk', 'is_family']].sort_values(by='is_family', ascending=False)
proxy = proxy.drop_duplicates(subset='party_rk', keep='first')
print(proxy.shape)

function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-test/family/stg_tb_policy_policyparticipant_202505220106.csv
Succesfully loaded data from: s3://tli-crm-segmentation/data-test/family/stg_tb_policy_policyparticipant_202505220106.csv
function: load_data successfully executed at 23.626342058181763s
function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-test/family/stg_tb_mstperson_beneficiary_party_id_name_bd_party_id_govt_202505220042.csv


/home/ec2-user/SageMaker/crm_pipeline/package/utils.py:119: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path, dtype=dtype)


Succesfully loaded data from: s3://tli-crm-segmentation/data-test/family/stg_tb_mstperson_beneficiary_party_id_name_bd_party_id_govt_202505220042.csv
function: load_data successfully executed at 30.933185577392578s
0          36935725
1          23012701
2          86577924
3          86951033
4          82274040
             ...   
4180703    90375835
4180704    91043748
4180705    91043749
4180706    91043753
4180707    91043754
Name: policy_no, Length: 4180708, dtype: object
26           6090194
27           3531868
29           2219125
35          12216290
40           1576143
              ...   
17950951      877950
17952137     8940559
17952138     9257906
17952140     3403441
17952144     9355729
Name: party_id, Length: 4180704, dtype: object
(5, 5)
(3324764, 2)


In [95]:
print(proxy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3324764 entries, 4115293 to 4180707
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   party_rk   object
 1   is_family  int64 
dtypes: int64(1), object(1)
memory usage: 76.1+ MB
None


In [89]:
beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_govt'] = beneid_df.loc[beneid_df['party_id_govt'].isna(), 'party_id_name_bd']
mask = ~beneid_df['party_id_govt'].isna()
mask &= beneid_df['relationshipcode'].isin([1, 2, 3, 4, 5, 22, 43, 44])
mask &= beneid_df['policyno'].isin(policyactive_list)
# mask &= beneid_df['party_id_govt'].isin(activebeneid_list)
beneidactive_df = beneid_df.loc[mask, :].head()

In [90]:
beneidactive_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 1452333 to 1452349
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   policytype            5 non-null      object 
 1   policyno              5 non-null      object 
 2   relationshipcode      5 non-null      int64  
 3   party_id_name_bd      0 non-null      object 
 4   party_id_govt         5 non-null      object 
 5   dup_party_id_name_bd  0 non-null      float64
 6   dup_party_id_govt     0 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 320.0+ bytes


In [71]:
familypolicyno_list = set(beneidactive_df['policyno'])

# get df having party_rk and family flag
proxy = policy_df[['party_rk', 'policy_no']].copy()
proxy['is_family'] = 0
proxy.loc[proxy['policy_no'].isin(familypolicyno_list), 'is_family'] = 1
proxy = proxy[['party_rk', 'is_family']].sort_values(by='is_family', ascending=False)
proxy = proxy.drop_duplicates(subset='party_rk', keep='first')
proxy

,party_rk,is_family
2345233,8006345838,1
2699221,8007273637,1
2371092,8006405212,1
2383704,8006435885,1
647843,8001683555,1
...,...,...
1393573,8003650843,0
1393575,8003650846,0
1393577,8003650851,0
1393578,8003650854,0


In [73]:
proxy.head(20)

,party_rk,is_family
2345233,8006345838,1
2699221,8007273637,1
2371092,8006405212,1
2383704,8006435885,1
647843,8001683555,1
2787141,8007513754,0
2787139,8007513753,0
2787136,8007513752,0
2787135,8007513750,0
2787133,8007513748,0


In [76]:
mask = ~beneidactive_df['party_id_govt'].isna()
mask = beneid_df.duplicated(subset='party_id_govt')
set_dup = set(beneid_df[mask]['policyno'])
active_list = set(policy_df['policy_no'])
print(f'interset id:{len(active_list&set_dup)}')

In [77]:
len(set_dup)

0

0

In [79]:
familyv1_df = pd.read_csv('s3://tli-crm-segmentation/data-zones/neutral/processed/family.csv')
familyv1_df = familyv1_df.drop_duplicates(subset='party_rk')
familyv1_df['is_family'].value_counts()

0    2728562
1     596202
Name: is_family, dtype: int64

In [91]:
proxy = proxy.drop_duplicates(subset='party_rk')
proxy['is_family'].value_counts()

0    3324759
1          5
Name: is_family, dtype: int64

In [13]:
familyv1_df = familyv1_df.sort_values(by='is_family', ascending=False)
familyv1_df = familyv1_df.drop_duplicates(subset='party_rk', keep='first')
familyv1_df

,party_rk,is_family
0,8000342016,1
2764959,8007454937,1
3567762,8009591575,1
3567760,8009591571,1
778968,8002045299,1
...,...,...
1588494,8004194570,0
1588495,8004194572,0
1588496,8004194573,0
1588497,8004194578,0


In [14]:
familyv1_df['is_family'].value_counts()

0    2728562
1     596202
Name: is_family, dtype: int64